In [ ]:
import pickle
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import torch
from modules.visualize import *
from modules.evaluate import *
from modules.utils import *

In [ ]:
dataset = pickle.load(open('data/dataset.pkl', 'rb'))
trx_orfs = pickle.load(open('data/trx_orfs.pkl', 'rb'))
ensembl_trx = pickle.load(open('data/ensembl_trx.pkl', 'rb'))

In [ ]:
tis_result = {}
# Loop through the CSV files
for i in range(6):
    # Read the CSV file into a dataframe
    df = pd.read_csv(f'preds/results{i}.csv')
    # Convert the dataframe to a list of dictionaries
    result = df.to_dict('records')
    
    # Loop through the list of dictionaries
    for attrs in result:
        trx = attrs['ID'].split(" ")[0].split("'")[1]
        start = attrs['TIS_pos']
        stop = attrs['TTS_pos']
        out = attrs['output']
        # Check if the trx already exists in the dictionary
        if trx not in tis_result:
            tis_result[trx] = {'start': start, 
                               'stop': stop, 
                               'out': out}
        elif trx in tis_result and out > tis_result[trx]['out']:
            tis_result[trx] = {'start': start, 
                               'stop': stop, 
                               'out': out}

In [ ]:
data_dict = {}

In [ ]:
data_array = np.load('preds/results0.npy', allow_pickle=True)
for i in range(data_array.shape[0]):
    trx = data_array[i][2][0].split(' ')[0]
    out = data_array[i][0][0]
    data_dict[trx] = out
data_array = np.load('preds/results1.npy', allow_pickle=True)
for i in range(data_array.shape[0]):
    trx = data_array[i][2][0].split(' ')[0]
    out = data_array[i][0][0]
    data_dict[trx] = out
data_array = np.load('preds/results2.npy', allow_pickle=True)
for i in range(data_array.shape[0]):
    trx = data_array[i][2][0].split(' ')[0]
    out = data_array[i][0][0]
    data_dict[trx] = out
data_array = np.load('preds/results3.npy', allow_pickle=True)
for i in range(data_array.shape[0]):
    trx = data_array[i][2][0].split(' ')[0]
    out = data_array[i][0][0]
    data_dict[trx] = out
data_array = np.load('preds/results4.npy', allow_pickle=True)
for i in range(data_array.shape[0]):
    trx = data_array[i][2][0].split(' ')[0]
    out = data_array[i][0][0]
    data_dict[trx] = out
data_array = np.load('preds/results5.npy', allow_pickle=True)
for i in range(data_array.shape[0]):
    trx = data_array[i][2][0].split(' ')[0]
    out = data_array[i][0][0]
    data_dict[trx] = out

In [ ]:
tis_imputed = dict()
for trx, output in data_dict.items():
    if trx not in tis_result:
        tis_imputed[trx] = output
    else:
        start = tis_result[trx]['start']
        stop = tis_result[trx]['stop']
        out = tis_result[trx]['out']
        output[start:stop] = out
        tis_imputed[trx] = output

In [ ]:
len(tis_imputed)

In [ ]:
reports = pickle.load(open('preds/reports.pkl', 'rb'))

In [23]:
tis_reports = dict()
for trx, attrs in reports.items():
    out = None
    if trx in tis_imputed:
        out = torch.tensor(tis_imputed[trx])
    else:
        out = torch.ones(len(ensembl_trx[trx]['sequence']))
    mapped_seq = attrs['mapped_seq']
    mapped_cds = attrs['mapped_cds']
    preds = bin_pred(out, 0.5)
    recall = recall_score(mapped_cds, preds)
    iou = iou_score(mapped_cds, preds)
    tis_reports[trx] = {'out':out,
                       'mapped_seq':mapped_seq,
                       'mapped_cds':mapped_cds,
                       'iou':iou,
                       'recall':recall}

In [27]:
pickle.dump(tis_reports, open('preds/tis_reports.pkl', 'wb'))